In [ ]:
```
1. Working on data downloaded from BINANCE by Justin
2. Split data into training and test
3. Fit the model(s) to the training data
4. Evaluate the trained model(s) by using testing data (including calculations, metrics, or visualizations needed to evaluate the performance).
5. Optionally, apply a dimensionality reduction technique to reduce the input features, or perform feature engineering to generate new features to train the model.
6. Show the predictions by using a sample of new data. Compare the predictions if more than one model is used.
7. Save PNG images of your visualizations to distribute to the class and instructional team, as well as to include in your presentation and your repo’s README.md file.
8. Use one new machine learning library, machine learning model, or evaluation metric that hasn't been covered in class.
9. Create a README.md in your repo with a write-up summarizing your project. Be sure to include any usage instructions to set up and use the model.
```

In [ ]:
#get eth daily candle from 2016-2022
candles = client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_1DAY, "1 Dec, 2016", "1 Dec, 2022")

In [ ]:
#set list to hold candle data
open = []
high = []
low = []
close = []
volume = []
time=[]

In [ ]:
candles

In [ ]:
#for loop that adds data to lists above
for i in range(len(candles)):
    open.append(candles[i][1])
    high.append(candles[i][2])
    low.append(candles[i][3])
    close.append(candles[i][4])
    volume.append(candles[i][5])
    time.append(candles[i][6])

In [ ]:
import pandas as pd

candle_df_open = pd.DataFrame(open, columns= ['open'])
candle_df_high = pd.DataFrame(high, columns= ['high'])
candle_df_low = pd.DataFrame(low, columns= ['low'])
candle_df_close = pd.DataFrame(close, columns= ['close'])
candle_df_volume = pd.DataFrame(volume, columns= ['volume'])
candle_df_time = pd.DataFrame(time, columns= ['time'])

In [ ]:
candle_df = pd.concat([candle_df_open,candle_df_high,candle_df_low,candle_df_close,candle_df_volume, candle_df_time], axis =1)

In [ ]:
candle_df.head()

In [ ]:
candle_df['time'] = pd.to_datetime(candle_df['time'], unit='ms')

candle_df['time']

In [ ]:
candle_df = candle_df.set_index('time')


In [ ]:
candle_df.head()

In [ ]:
candle_df.to_csv('C:/Users/Justin/Rutgers/project-2/Resources/eth.csv')

In [ ]:
candle_df = candle_df.astype('float')

In [ ]:
signals_df = candle_df.loc[:,['close']]

signals_df['actual returns'] = signals_df['close'].pct_change()

signals_df = signals_df.dropna()

display(signals_df.head())
display(signals_df.tail())

In [ ]:
from finta import TA

In [ ]:
signals_df['RSI'] = TA.AO(candle_df)
#signals_df['STOCH'] = TA.BBWIDTH(candle_df)

In [ ]:
signals_df.head(70)

In [ ]:
# Initialize the new Signal column
signals_df['Signal'] = 0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['RSI'] > 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['RSI'] < 0) , 'Signal'] = -1

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
#signals_df.loc[(signals_df['RSI'] <= 35) & (signals_df['STOCH'] < .25 ), 'Signal'] = 1
# When Actual Returns are less than 0, generate signal to sell stock short
#ignals_df.loc[(signals_df['RSI'] >= 60) & (signals_df['STOCH'] < .25 ), 'Signal'] = -1

In [ ]:
display(signals_df.head())
display(signals_df.tail())

In [ ]:
signals_df['Signal'].value_counts()

In [ ]:
signals_df['Strategy Returns'] = signals_df['actual returns'] * signals_df['Signal'].shift().dropna()

In [ ]:
display(signals_df.head())
display(signals_df.tail())

In [ ]:
(1 + signals_df['Strategy Returns']).cumprod().plot()

In [ ]:
X = signals_df[['RSI']].shift().dropna()

y = signals_df['Signal']

# Review the value counts
y.value_counts()

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [ ]:
training_end = X.index.min() + DateOffset(months=12)

# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(months=6):]
y_test = y.loc[training_end+DateOffset(months=6):]

# Review the X_test DataFrame
X_train.head()

In [ ]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# From SVM, instantiate SVC classifier model instance
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
 
# Fit the model to the data using the training data
lr_model = lr_model.fit(X_train_scaled,y_train)
 
# Use the testing data to make the model predictions
lr_pred = lr_model.predict(X_test_scaled)

# Review the model's predicted values
lr_pred

In [ ]:
lr_testing_report = classification_report(y_test, lr_pred)

# Print the classification report
print(lr_testing_report)

In [ ]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = lr_pred

# Add the actual returns to the DataFrame
predictions_df['actual returns'] = signals_df['actual returns']

# Add the strategy returns to the DataFrame
predictions_df['Trading Algo Returns'] = signals_df['actual returns'] * predictions_df['Predicted'].shift()

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

In [ ]:
(1 + predictions_df[["actual returns", "Trading Algo Returns"]]).cumprod().plot()

In [ ]:
tbill_df = pd.read_csv('C:/Users/Justin/Rutgers/project-2/Resources/DGS2 (2).csv')

In [ ]:
tbill_df = tbill_df.set_index('DATE')


In [ ]:
new_signals_df = pd.concat([candle_df,tbill_df])
asd



In [ ]:
candle_df

In [ ]:
tbill_df